In [1]:
import os
import sys
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


In [2]:
def init_spark():
    spark = SparkSession.builder.appName("Python Spark SQL basic example").config("spark.some.config.option", "some-value").getOrCreate()
    return spark

In [3]:
spark_object = init_spark()
train_identity = spark_object.read.option("delimiter", ",").csv("./data/train_identity.csv", header='true', inferSchema='true')
train_transaction = spark_object.read.option("delimiter", ",").csv("./data/train_transaction.csv", header='true', inferSchema='true')

23/03/14 08:29:39 WARN Utils: Your hostname, karyubuntu resolves to a loopback address: 127.0.1.1; using 192.168.0.165 instead (on interface wlp0s20f3)
23/03/14 08:29:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/14 08:29:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
print(train_identity.count())
print(train_transaction.count())
print(train_identity.head())
print(train_transaction.head())

144233
590540
23/03/13 23:52:19 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
Row(TransactionID=2987004, id_01=0.0, id_02=70787.0, id_03=None, id_04=None, id_05=None, id_06=None, id_07=None, id_08=None, id_09=None, id_10=None, id_11=100.0, id_12='NotFound', id_13=None, id_14=-480.0, id_15='New', id_16='NotFound', id_17=166.0, id_18=None, id_19=542.0, id_20=144.0, id_21=None, id_22=None, id_23=None, id_24=None, id_25=None, id_26=None, id_27=None, id_28='New', id_29='NotFound', id_30='Android 7.0', id_31='samsung browser 6.2', id_32=32.0, id_33='2220x1080', id_34='match_status:2', id_35='T', id_36='F', id_37='T', id_38='T', DeviceType='mobile', DeviceInfo='SAMSUNG SM-G892A Build/NRD90M')
Row(TransactionID=2987000, isFraud=0, TransactionDT=86400, TransactionAmt=68.5, ProductCD='W', card1=13926, card2=None, card3=150.0, card4='discover', card5=142.0, card6='credit', addr1=315

In [4]:
train = train_transaction.join(train_identity, on="TransactionID", how='inner')

In [6]:
print(train.count())
train.head()

144233


Row(TransactionID=2987048, isFraud=0, TransactionDT=87317, TransactionAmt=42.294, ProductCD='C', card1=15885, card2=545.0, card3=185.0, card4='visa', card5=138.0, card6='debit', addr1=None, addr2=None, dist1=None, dist2=None, P_emaildomain='outlook.com', R_emaildomain='outlook.com', C1=1.0, C2=1.0, C3=0.0, C4=1.0, C5=0.0, C6=1.0, C7=1.0, C8=1.0, C9=0.0, C10=1.0, C11=1.0, C12=1.0, C13=1.0, C14=1.0, D1=0.0, D2=None, D3=None, D4=0.0, D5=None, D6=0.0, D7=None, D8=None, D9=None, D10=0.0, D11=None, D12=0.0, D13=0.0, D14=0.0, D15=0.0, M1=None, M2=None, M3=None, M4='M2', M5=None, M6=None, M7=None, M8=None, M9=None, V1=None, V2=None, V3=None, V4=None, V5=None, V6=None, V7=None, V8=None, V9=None, V10=None, V11=None, V12=0.0, V13=0.0, V14=1.0, V15=1.0, V16=1.0, V17=1.0, V18=1.0, V19=1.0, V20=1.0, V21=1.0, V22=1.0, V23=1.0, V24=1.0, V25=1.0, V26=1.0, V27=0.0, V28=0.0, V29=0.0, V30=0.0, V31=1.0, V32=1.0, V33=1.0, V34=1.0, V35=0.0, V36=0.0, V37=1.0, V38=1.0, V39=1.0, V40=1.0, V41=1.0, V42=1.0, V43=1

In [7]:
total_count = train.count()
def null_percentage(df, column):
    null_count = df.where(col(column).isNull()).count()
    return null_count / total_count

 
for column in train.columns:
    if null_percentage(train, column) > 0.05:
        train = train.drop(col(column))

In [8]:
print((train.count(), len(train.columns)))
train.toPandas().to_csv('./data/traincleaned.csv')

(144233, 194)


load data from train cleaned file 

In [26]:
train = spark_object.read.option("delimiter", ",").csv("./data/traincleaned.csv", header='true', inferSchema='true')
train = train.drop(col("_c0"))

In [27]:

li = (train.columns)

work with test data

In [28]:
test_identity = spark_object.read.option("delimiter", ",").csv("./data/test_identity.csv", header='true', inferSchema='true')
test_transaction = spark_object.read.option("delimiter", ",").csv("./data/test_transaction.csv", header='true', inferSchema='true')

In [33]:
print(test_identity.columns)
print(train_identity.columns)
print(test_transaction.columns)
print(test_transaction.columns)

['TransactionID', 'id-01', 'id-02', 'id-03', 'id-04', 'id-05', 'id-06', 'id-07', 'id-08', 'id-09', 'id-10', 'id-11', 'id-12', 'id-13', 'id-14', 'id-15', 'id-16', 'id-17', 'id-18', 'id-19', 'id-20', 'id-21', 'id-22', 'id-23', 'id-24', 'id-25', 'id-26', 'id-27', 'id-28', 'id-29', 'id-30', 'id-31', 'id-32', 'id-33', 'id-34', 'id-35', 'id-36', 'id-37', 'id-38', 'DeviceType', 'DeviceInfo']
['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']
['TransactionID', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'

In [29]:
test = test_transaction.join(test_identity, on="TransactionID", how='inner')
for name in test.columns:
    new_name = name.replace('-', '_')
    if (new_name != name):
        test = test.withColumnRenamed(name,new_name)

In [30]:
for column in test.columns:
    if column not in li:
        test = test.drop(col(column))

In [35]:
print((test.count(), len(test.columns)))
test.toPandas().to_csv('./data/testcleaned.csv')

(141907, 193)
23/03/14 09:17:28 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


load cleaned test data

In [36]:
test = spark_object.read.option("delimiter", ",").csv("./data/testcleaned.csv", header='true', inferSchema='true')
test = test.drop(col("_c0"))